---
title: "Portland Crime Analysis"
subtitle: "Data Science Portfolio"
author: "Tucker Trost"
format:
  html:
    self-contained: true
    page-layout: full
    title-block-banner: true
    toc: true
    toc-depth: 3
    toc-location: body
    number-sections: false
    html-math-method: katex
    code-fold: true
    code-summary: "Show the code"
    code-overflow: wrap
    code-copy: hover
    code-tools:
        source: false
        toggle: true
        caption: See code
execute: 
  warning: false
    
---




## Portland Crime Analysis

## Elevator pitch

_This project involves cleaning and exploring crime data from Portland, Oregon (2016-2023). After addressing missing data and unnecessary columns, we uncovered key trends in offense types, report times, and their relationship to neighborhood, day, and time. These insights help provide a clearer picture of crime patterns in the city._

### Data Source
_The dateset that I am using was accessed directly from [Portland Police Bureau's Open Data initiative](https://www.portland.gov/police/open-data/crime-statistics); compiled from 2015-2023. This dataset is being used under this [license](https://creativecommons.org/publicdomain/zero/1.0/)._

### Data Dictionary

_**Address:** Address of reported incident at the 100 block level (e.g.: 1111 SW 2nd Ave would be 1100 Block SW 2nd Ave)._

_**Case Number:** The case year and number for the reported incident (YY-######)._

_**Crime Against:** Crime against category (Person, Property, or Society)._

_**Neighborhood:** Neighborhood where incident occurred. If the neighborhood name is missing, the incident occurred outside of the boundaries of the Portland neighborhoods or at a location that could not be assigned to a specific address in the system. (e.g., Portland, near Washington Park, on the streetcar, etc.)._

_**Occur Date:** Date the incident occurred. The exact occur date is sometimes unknown. In most situations, the first possible date the crime could have occurred is used as the occur date. (For example, victims return home from a week-long vacation to find their home burglarized. The burglary could have occurred at any point during the week. The first date of their vacation would be listed as the occur date.)_

_**Occur Time**: Time the incident occurred. The exact occur time is sometimes unknown. In most situations, the first possible time the crime could have occurred is used as the occur time. The time is reported in the 24-hour clock format, with the first two digits representing hour (ranges from 00 to 23) and the second two digits representing minutes (ranges from 00 to 59)._

_**Offense Category:** Category of offense (for example, Assault Offenses)._

_**Offense Type:** Type of offense (for example, Aggravated Assault)Note: The statistic for Homicide Offenses has been updated in the Group A Crimes report to align with the 2019 FBI NIBRS definitions. The statistic for Homicide Offenses includes (09A) Murder & Non-negligent Manslaughter and (09B) Negligent Manslaughter. As of January 1, 2019, the FBI expanded the definition of negligent manslaughter to include traffic fatalities that result in an arrest for driving under the influence, distracted driving, or reckless driving. **The change in definition impacts the 2019 homicide offenses statistic and the comparability of 2019 homicide statistics to prior year.**_

_**Open Data Lat/Lon:** Generalized Latitude / Longitude of the reported incident. For offenses that occurred at a specific address, the point is mapped to the block's midpoint. Offenses that occurred at an intersection is mapped to the intersection centroid._

_**Open Data X/Y:** Generalized XY point of the reported incident. For offenses that occurred at a specific address, the point is mapped to the block's midpoint. Offenses that occurred at an intersection is mapped to the intersection centroid. To protect the identity of victims and other privacy concerns, the points of certain case types are not released. XY points use the Oregon State Plane North (3601), NAD83 HARN, US International Feet coordinate system._

_**Offense Count:** Number of offenses per incident. Offenses (i.e. this field) are summed for counting purposes._


In [ ]:
#| label: libraries
#| include: false
import pandas as pd
import numpy as np
import datetime as dt
# from plotnine import *
from lets_plot import *
LetsPlot.setup_html()


## Prepare


In [ ]:
#| label: project-data
#| code-summary: Read and format project data
# Load in Data
pcrime_15 = pd.read_csv('CrimeData-2015.csv')
pcrime_16 = pd.read_csv('CrimeData-2016.csv')
pcrime_17 = pd.read_csv('CrimeData-2017.csv')
pcrime_18 = pd.read_csv('CrimeData-2018.csv')
pcrime_19 = pd.read_csv('CrimeData-2019.csv')
pcrime_20 = pd.read_csv('CrimeData-2020.csv')
pcrime_21 = pd.read_csv('CrimeData-2021.csv')
pcrime_22 = pd.read_csv('CrimeData-2022.csv')
pcrime_23 = pd.read_csv('CrimeData-2023.csv')

# Combine Datasets
pcrime_combined = pd.concat([pcrime_15,pcrime_16,pcrime_17,pcrime_18,pcrime_19,pcrime_20,pcrime_21,pcrime_22,pcrime_23], ignore_index=True)
pcrime_combined.head()

### What is our goal?
_Thinking forward to the analysis that I want to perform with this data, I need to understand what I am looking for when it comes to cleaning. I know that I want to focus my analysis on the distribution of different offense types and categories across the various neighborhoods of Portland. Additionally, I'd like to get insight into the temporal trends that lie within the data. Based on this understanding, I get a better sense of what aspects of the data need to be cleaned._


In [ ]:
#| label: Prepare
#| code-summary: Identify Missing Data

pcrime_combined.isna().sum()

### Initial Observations

1. ***A time to report column would be useful***
    - *Convert OccurDate and ReportDate to datetime*
    - *Create a time to report column*

2. ***OpenDataX/Y don't seem necessary for our analysis***
    - *Drop OpenDataX and OpenDataY columns*

3. ***Address column seems to be redundant as most entries are just a general location***
    - *Drop Address column*

4. ***Neighborhood averages can be used to find lat/lon***
    - *Drop rows with missing Neighborhood and OpenDataLat*
    - *Replace all rows with neighborhood but missing Lat/Lon data with average Lat/Lon of their neighborhood*




## Data Cleaning


In [ ]:
#| label: data-cleaning
#| code-summary: Cleaning

# Calculate average Lat/Lon for each neighborhood
neighborhood_means = pcrime_combined.groupby('Neighborhood')[['OpenDataLat','OpenDataLon']].transform('mean')

# Clean the data
pcrime_cleaned = (
    pcrime_combined
    .drop(columns=['Address', 'OpenDataX', 'OpenDataY'])  # Drop X/Y
    .dropna(subset=['OpenDataLat', 'Neighborhood'], how='all')  # Drop missing lat/lon and Neighborhoods
    .assign(
        OccurDate=pd.to_datetime(pcrime_combined['OccurDate']),  # Convert dates to datetime
        week=lambda x: x.OccurDate.dt.isocalendar().week,
        year=lambda x: x.OccurDate.dt.year,
        month=lambda x: x.OccurDate.dt.month,
        dayofmonth=lambda x: x.OccurDate.dt.day,
        ReportDate=pd.to_datetime(pcrime_combined['ReportDate']),
        ReportDiff=lambda x: (x['ReportDate'] - x['OccurDate']).dt.days,  # Calculate time to report
        OpenDataLat=lambda x: x['OpenDataLat'].fillna(neighborhood_means['OpenDataLat']),  # Fill missing Lat/Lon with average Lat/Lon of given neighborhood
        OpenDataLon=lambda x: x['OpenDataLon'].fillna(neighborhood_means['OpenDataLon']),
        OccurTime=lambda x: x['OccurTime'].astype(str).str.zfill(4),  # Ensure time is in HHMM format
        OccurDateTime=lambda x: pd.to_datetime(
            x['OccurDate'].dt.strftime('%Y-%m-%d') + ' ' + 
            x['OccurTime'].str[:2] + ':' + x['OccurTime'].str[2:]), # Combine date and formatted time into datetime
        OccurHour=lambda x: x.OccurDateTime.dt.hour,
    )
    .loc[lambda x: x['OccurDateTime'].dt.year.between(2015, 2023)]  # Filter rows with years within 2015–2023
)

pcrime_cleaned

In [ ]:
pcrime_cleaned.info()

_Now we can check and see how we did filling in our missing data._


In [ ]:
#| label: data-cleaning-2
#| code-summary: Check missing data

pcrime_cleaned.isna().sum()

### Final Cleaning Thoughts

_We have now cleaned our data into a useable state for our analysis. We went from many missing rows from in many columns to only 7881 missing rows in the neighborhood column. Since we have all of the Latitude and Longitude data for each of these missing rows, the missing data will still be useable for our visualizations in Tableau._

_Note: Further cleaning of the missing neighborhood rows could be done using a reverse geocoding API, however, that is beyond the scope of this project_

## Data Exploration

### Thoughts

* Temporal Trends
    - hour/Day/month/year trends
* Crime distributions
    - categories/types/crimeagainst
    - reportdiff by offense type
* Neighborhood
    - type/category distributions
    - reportdiff by neighborhood
    - offense count by neighborhood

#### Temporal Trends


In [ ]:
#| label: data-explore-1
#| code-summary: year-count

from lets_plot import *
LetsPlot.setup_html()
month_year = (pcrime_cleaned
    .assign(
        year=pcrime_cleaned['OccurDateTime'].dt.year,
        month=pcrime_cleaned['OccurDateTime'].dt.month) 
    .query('year >= 2019')\
    .groupby(['year', 'month'])\
    ['OffenseCount'].sum()\
    .reset_index()\
    .rename(columns={'OffenseCount': 'count'})
)

last_month_data = (
    month_year.groupby("year")
    .apply(lambda df: df[df["month"] == df["month"].max()])
    .reset_index(drop=True)
)

month_year["year"] = month_year["year"].astype(str)  # Convert year to string
last_month_data["year"] = last_month_data["year"].astype(str)  # Convert for labels

month_year_fig = (
    ggplot(month_year, aes(x="month", y="count", color="year", group="year")) +
    geom_smooth(method='loess', span=0.5,se=False) +
    geom_label(data=last_month_data, mapping=aes(y='count',label="year", color="year"),x=12.5, size=7,check_overlap=True)
 +  # Equivalent of geom_label_repel
    labs(title="Offense Counts by Month and Year", x="Month", y="Offense Count") +
    theme_minimal() +
    theme(legend_position="none") +
    scale_x_continuous(breaks=list(range(1, 13)))+
    scale_y_continuous(breaks=[4500,4750,5000,5250,5500,5750,6000,6250,6500]))


month_year_fig

##### Observations
This chart shows that crime levels remained fairly consistent in 2019. In 2020, we see a noticeable drop when the country went into lockdown, followed by a sharp increase as restrictions eased in the summer. Then, in 2021, Portland experienced a significant surge in crime, which remained relatively high until 2023, when it began to stabilize.


In [ ]:
#| label: data-explore-2
#| code-summary: Temporal Counts

# Month Count

month_count = pcrime_cleaned.groupby(pcrime_cleaned['OccurDateTime'].dt.month)['OffenseCount'].sum().reset_index()

month_count.rename(columns={month_count.columns[0]: 'Month'}, inplace=True)

month_count_fig = ggplot(month_count, aes(x="Month", y="OffenseCount")) + \
    geom_line(color='#2e6f40', size=1.5) + \
    geom_point(color='#2e6f40', size=3) + \
    labs(title='Offense Counts by Month', x='Month', y='Offense Count') + \
    scale_x_continuous(breaks=list(range(1, 13))) + \
    theme_minimal() + \
    theme(
        plot_title=element_text(size=16, face='bold'),
        axis_title_x=element_text(size=12, face='bold'),
        axis_title_y=element_text(size=12, face='bold'),
        axis_text_x=element_text(size=10), 
        axis_text_y=element_text(size=10),
        panel_grid_minor=element_blank()
    )

# Weekday Count
weekday_count = pcrime_cleaned.groupby(pcrime_cleaned['OccurDateTime'].dt.weekday)['OffenseCount'].sum().reset_index()

weekday_count.rename(columns={weekday_count.columns[0]: 'Weekday'}, inplace=True)

weekday_count_fig = ggplot(weekday_count, aes(x="Weekday", y="OffenseCount")) + \
    geom_line(color='#2e6f40', size=1.5) + \
    geom_point(color='#2e6f40', size=3) + \
    labs(title='Offense Counts by Weekday', x='Weekday', y='Offense Count') + \
    theme_minimal() + \
    theme(
        plot_title=element_text(size=16, face='bold'),
        axis_title_x=element_text(size=12, face='bold'),
        axis_title_y=element_text(size=12, face='bold'),
        axis_text_x=element_text(size=10), 
        axis_text_y=element_text(size=10),
        panel_grid_minor=element_blank()
    )

# Hour Count
hour_count = pcrime_cleaned.groupby(pcrime_cleaned['OccurDateTime'].dt.hour)['OffenseCount'].sum().reset_index()

hour_count.rename(columns={hour_count.columns[0]: 'Hour'}, inplace=True)

hour_count_fig = ggplot(hour_count, aes(x="Hour", y="OffenseCount")) + \
    geom_line(color='#2e6f40', size=1.5) + \
    geom_point(color='#2e6f40', size=3) + \
    labs(title='Offense Counts by Hour', x='Hour', y='Offense Count') + \
    scale_x_continuous(breaks=list(range(0, 25))) + \
    theme_minimal() + \
    theme(
        plot_title=element_text(size=16, face='bold'),
        axis_title_x=element_text(size=12, face='bold'),
        axis_title_y=element_text(size=12, face='bold'),
        axis_text_x=element_text(size=10), 
        axis_text_y=element_text(size=10),
        panel_grid_minor=element_blank()
    )

# Create a list of plots to display in a single row
temp_plot_list = [
    month_count_fig,
    weekday_count_fig,
    hour_count_fig
]

# Arrange the plots in a single row
temp_plots = gggrid(temp_plot_list, ncol=3) + ggsize(1200, 400)

# Show the combined plot
temp_plots

##### Observations
Crime patterns exhibit distinct temporal trends across months, weekdays, and hours. Monthly data shows that crime tends to slow down during the winter months and gradually rises through the summer and into the rest of the year, potentially influenced by seasonal factors such as weather and increased outdoor activity. Looking at weekly patterns, Friday stands out as the day with the highest number of reported offenses, which aligns with the start of the weekend when more people are out, creating more opportunities for crime. Additionally, crime follows a predictable daily cycle, with certain hours experiencing higher offense counts. These trends suggest that external factors like weather, social behavior, and law enforcement presence may play a role in crime fluctuations, warranting further analysis to uncover deeper insights.

#### Crime Distribution Trends


In [ ]:
#| label: data-explore-5
#| code-summary: crime_against_count

# Crime Against Count
crime_against = pcrime_cleaned.groupby('CrimeAgainst',as_index=False)['OffenseCount'].sum()
crime_against_fig = ggplot(crime_against, aes(x='CrimeAgainst', y='OffenseCount')) + \
    geom_bar(stat='identity', fill='#2e6f40', color='black') + \
    labs(title='Offense Counts by Crime Type', x='Crime Against', y='Offense Count') + \
    theme_minimal() + \
    theme(
        plot_title=element_text(size=16, face='bold'),
        axis_title_x=element_text(size=12, face='bold'),
        axis_title_y=element_text(size=12, face='bold'),
        axis_text_x=element_text(size=10), 
        axis_text_y=element_text(size=10),
        panel_grid_minor=element_blank()
    )

# Crime Against Report Time

crime_against_report_box = (ggplot(pcrime_cleaned,aes(x='ReportDiff',y='CrimeAgainst'))+
  geom_boxplot(outlier_shape = None,fill='#2e6f40', color='black')+
  scale_x_log10()+
  theme_minimal()+
  labs(title = 'Crime Against Report Time Distribution',x= 'Days to Report',y='Crime Against'))

crime_against_list = [crime_against_fig,crime_against_report_box]

crime_against_plots = gggrid(crime_against_list,ncol=2)+ ggsize(1600, 600)

crime_against_plots

##### Observations

It’s evident that crimes against property are much more common than other categories. My initial thought is that property crimes may be more frequent because they’re often easier to commit, both physically and morally. Property doesn’t involve direct harm to individuals, which could make it feel less risky or less severe to potential offenders. The differences in report times are also interesting. All crime against types have a median report time of 1 day, however, crime against person has a larger distribution of report times. This also makes sense because many crimes against a person are very sensitive situations that lead to delayed reporting.


In [ ]:
#| label: data-explore-7
#| code-summary: crime_category_count

# Crime Category Count
crime_cat = pcrime_cleaned.groupby('OffenseCategory',as_index=False)['OffenseCount'].sum().sort_values(by='OffenseCount', ascending=False).head(10)
crime_cat_fig = ggplot(crime_cat, aes(x='OffenseCount', y='OffenseCategory')) + \
    geom_bar(stat='identity', fill='#2e6f40', color='black') + \
    labs(title='Offense Counts by Category', x='Offense COunt', y='Offense Category') + \
    theme_minimal() + \
    theme(
        plot_title=element_text(size=16, face='bold'),
        axis_title_x=element_text(size=12, face='bold'),
        axis_title_y=element_text(size=12, face='bold'),
        axis_text_x=element_text(size=10), 
        axis_text_y=element_text(size=10),
        panel_grid_minor=element_blank()
    )

crime_cat_fig

# Crime Type Count
crime_type = pcrime_cleaned.groupby('OffenseType',as_index=False)['OffenseCount'].sum().sort_values(by='OffenseCount', ascending=False).head(10)
crime_type_fig = ggplot(crime_type, aes(x='OffenseCount', y='OffenseType')) + \
    geom_bar(stat='identity', fill='#2e6f40', color='black') + \
    labs(title='Offense Counts by Type', x='Offense COunt', y='Offense Type') + \
    theme_minimal() + \
    theme(
        plot_title=element_text(size=16, face='bold'),
        axis_title_x=element_text(size=12, face='bold'),
        axis_title_y=element_text(size=12, face='bold'),
        axis_text_x=element_text(size=10), 
        axis_text_y=element_text(size=10),
        panel_grid_minor=element_blank()
    )

crime_type_fig

# Neighborhood Count
neigh_count = pcrime_cleaned.groupby('Neighborhood',as_index=False)['OffenseCount'].sum().sort_values(by='OffenseCount', ascending=False).head(10)
neigh_count_fig = ggplot(neigh_count,aes(y=neigh_count["Neighborhood"],x=neigh_count["OffenseCount"]))+ \
    geom_bar(stat='identity',fill='#2e6f40', color='black')+ \
    labs(title='Offense Count by Neighborhood', x='Offense Count', y='Neighborhood') + \
    theme_minimal() + \
    theme(
        plot_title=element_text(size=16, face='bold'),
        axis_title_x=element_text(size=12, face='bold'),
        axis_title_y=element_text(size=12, face='bold'),
        axis_text_x=element_text(size=10), 
        axis_text_y=element_text(size=10),
        panel_grid_minor=element_blank()
    )
neigh_count_fig

count_list = [crime_cat_fig,crime_type_fig,neigh_count_fig]

count_plots = gggrid(count_list, ncol=2) + ggsize(1200, 400)

count_plots

##### Observations
Larceny stands out as the most common offense in Portland, reinforcing the broader trend that property crimes are significantly more prevalent than other crime categories. This may be attributed to the opportunistic nature of larceny—these offenses often require little planning and can happen quickly, unlike more serious crimes that demand time, effort, or emotional involvement. Additionally, neighborhood-level analysis shows that Downtown and Hazelwood experience disproportionately high numbers of reported offenses. While this highlights potential crime hotspots, further context—such as population density, neighborhood size, and visitor traffic—would provide a more accurate understanding of these patterns.

## Current Final Observations

This project is still a work in progress, and there’s plenty more to explore and visualize to better address some of the questions raised during my analysis. That said, the work completed so far has revealed some fascinating insights into crime patterns in Portland, Oregon. Thank you for taking the time to check out my project—I hope you’ll return to see the updates as it continues to develop!

# Machine Learning


In [ ]:
def hour_bin(hour):
    if 5 <= hour < 12:
        return "Morning"
    elif 12 <= hour < 17:
        return "Afternoon"
    elif 17 <= hour < 21:
        return "Evening"
    else:
        return "Night"

def season_bin(month):
    if 3 <= month <= 5:
        return "Spring"
    elif 6 <= month <= 8:
        return "Summer"
    elif 9 <= month <= 11:
        return "Fall"
    else:
        return "Winter"

def weekday_bin(weekday):
    if weekday < 5:
        return "Weekday"
    else:
        return "Weekend"

def commute_hours(hour):
    if (6 <= hour < 9) or (16 <= hour < 19):
        return "Commuting Hours"
    else:
        return "Non-Commuting Hours"

def temp_bin(temp):
    if temp < 32:
        return "Freezing"
    elif 32 <= temp < 50:
        return "Cold"
    elif 50 <= temp < 68:
        return "Cool"
    elif 68 <= temp < 85:
        return "Warm"
    else:
        return "Hot"

def rainy_day(precipitation):
    if precipitation > 0:
        return "Rainy Day"
    else:
        return "Not Rainy Day"

In [ ]:
def assign_quadrant(neighborhood):
    north = [
        "Arbor Lodge", "Boise", "Bridgeton", "Cathedral Park", "East Columbia", "Eliot",
        "Hayden Island", "Humboldt", "Kenton", "Overlook", "Piedmont", "Portsmouth",
        "St. Johns", "St Johns", "University Park"
    ]

    northwest = [
        "Arlington Heights", "Forest Park", "Hillside", "Linnton", "Northwest Heights",
        "NW 21st & 23rd Avenue – Nob Hill", "Sylvan Highlands", "Sylvan-Highlands",
        "Northwest Industrial", "Northwest"
    ]

    northeast = [
        "Alameda", "Alberta Arts District", "Beaumont-Wilshire", "North Tabor", "Concordia",
        "Cully", "Grant Park", "Hollywood", "Irvington", "King", "Madison South",
        "Rose City Park", "Roseway", "Sabin", "Sullivan’s Gulch", "Sullivan's Gulch",
        "Sumner", "Sunderland", "Vernon", "Woodland Park", "Woodlawn", "Lloyd"
    ]

    central = [
        "China Town/Old Town", "Old Town/Chinatown", "Downtown", "Lloyd Center",
        "Pearl District", "Pearl", "South Waterfront"
    ]

    southwest = [
        "Arnold Creek", "Ashcreek", "Bridlemile", "Collins View", "Crestwood", "Far Southwest",
        "Goose Hollow", "Hayhurst", "Healy Heights", "Hillsdale", "Homestead", "Maplewood",
        "Markham", "Marshall Park", "Multnomah", "South Burlingame", "Southwest Hills",
        "West Portland Park", "South Portland"
    ]

    southeast = [
        "Belmont", "Brentwood-Darlington", "Brooklyn", "Buckman", "Buckman East", "Buckman West",
        "Creston-Kenilworth", "Eastmoreland", "Foster-Powell", "Hawthorne", "Hosford-Abernethy",
        "Johnson Creek (aka Ardenwald)", "Kerns", "Laurelhurst", "Montavilla", "Mount Scott",
        "Mount Tabor", "Mt Tabor", "Mt Scott-Arleta", "Reed", "Richmond", "Sellwood-Moreland",
        "South Tabor", "Sunnyside", "Woodstock", "Argay", "Centennial", "Glenfair", "Hazelwood",
        "Lents", "Mill Park", "Parkrose", "Parkrose Heights", "Pleasant Valley",
        "Powellhurst Gilbert", "Powellhurst-Gilbert", "Russell", "Wilkes", "Ardenwald"
    ]

    if neighborhood in north:
        return "North Portland"
    elif neighborhood in northwest:
        return "Northwest Portland"
    elif neighborhood in northeast:
        return "Northeast Portland"
    elif neighborhood in central:
        return "Central Portland"
    elif neighborhood in southwest:
        return "Southwest Portland"
    elif neighborhood in southeast:
        return "Southeast Portland"
    else:
        return "Unknown Quadrant"


In [ ]:
def fix_neigh(neighborhood):
    if neighborhood == 'SUMNER*':
        return 'SUMNER'
    elif neighborhood == 'NORTHWEST DISTRICT':
        return 'NORTHWEST'
    elif neighborhood == 'POWELLHURST GILBERT':
        return 'POWELLHURST-GILBERT'
    elif neighborhood == 'LLOYD DISTRICT':
        return 'LLOYD'
    elif neighborhood == 'SOUTHWEST HILLS*':
        return 'SOUTHWEST HILLS'
    elif neighborhood == 'ST. JOHNS':
        return 'ST JOHNS'
    elif neighborhood == 'OLD TOWN-CHINATOWN':
        return 'OLD TOWN/CHINATOWN'
    elif neighborhood == 'ARDENWALD-JOHNSON CREEK':
        return 'ARDENWALD'
    elif neighborhood == 'FOREST PARK*':
        return 'FOREST PARK'
    elif neighborhood == 'LINNTON*':
        return 'LINNTON'
    elif neighborhood == 'PLEASANT VALLEY*':
        return 'PLEASANT VALLEY'
    elif neighborhood == 'BRIDLEMILE*':
        return 'BRIDLEMILE'
    elif neighborhood == 'SYLVAN HIGHLANDS*':
        return 'SYLVAN-HIGHLANDS'
    else:
        return neighborhood

In [ ]:
pdata = pd.read_csv('pdata.csv')
pdata_cleaned = pdata.drop(columns=['Median 2021 home sale price ($)',
       'Average 2021 home sale price ($)', 'Homes sold in 2021 (#)',
       '1-year median price change 2020-21 (%)',
       '5-year median price change 2016-21 (%)', 'Days on market (average)',
       'Condo sales among home sales (#)',
       'Share of 2021 sales that were condos (%)', 'Short sales',
       'Bank-owned sales', 'Average square feet of homes sold',
       'Average cost per square foot ($)', 'Average year built of homes sold',
       "Renters' median monthly housing ($)",'Total newcomers (%)',
       'Assault', 'Homicide', 'Sex offenses', 'Arson', 'Robbery', 'Burglary','Larceny',
       'Motor vehicle thefts', 'Vandalism', 'Drug/narcotic offenses',
       'Violent crimes per 1,000 residents',
       'Burglary/larceny/vehicle theft crimes per 1,000 residents',
       'Vehicle thefts per square mile'])\
        .assign(
            Neighborhood = lambda x: x.Neighborhood.apply(fix_neigh)
        ).replace('-', np.nan)

In [ ]:
pdata_cleaned.columns

In [ ]:
from pandas.tseries.holiday import USFederalHolidayCalendar
calendar = USFederalHolidayCalendar()
holidays = calendar.holidays(start=pcrime_cleaned.OccurDate.min(), end=pcrime_cleaned.OccurDate.max())

In [ ]:
pcrime_ml = pcrime_cleaned\
    .groupby(['OccurDate','Neighborhood','OccurHour'])\
    .agg(
        total_offenses=('OffenseCount','sum'),
        lat=('OpenDataLat','mean'),
        lon=('OpenDataLon','mean'))\
    .reset_index()\
    .assign(
        OccurDate = lambda x: pd.to_datetime(x.OccurDate),
        week=lambda x: x.OccurDate.dt.isocalendar().week,
        year=lambda x: x.OccurDate.dt.year,
        month=lambda x: x.OccurDate.dt.month,
        month_sin=lambda x: np.sin(2 * np.pi * x.month / 12),
        month_cos=lambda x: np.cos(2 * np.pi * x.month / 12),
        quarter=lambda x: x.OccurDate.dt.quarter,
        weekday=lambda x: x.OccurDate.dt.weekday,
        weekday_sin=lambda x: np.sin(2 * np.pi * x.weekday / 7),
        weekday_cos=lambda x: np.cos(2 * np.pi * x.weekday / 7),
        hour_bin=lambda x: x['OccurHour'].apply(hour_bin),
        hour_sin=lambda x: np.sin(2 * np.pi * x.OccurHour / 24),
        hour_cos=lambda x: np.cos(2 * np.pi * x.OccurHour / 24),
        season_bin=lambda x: x['month'].apply(season_bin),
        weekday_bin=lambda x: x['weekday'].apply(weekday_bin),
        commute_hours=lambda x: x['OccurHour'].apply(commute_hours),
        holiday=lambda x: x.OccurDate.isin(holidays),
        covid_time = lambda x: (x.OccurDate.dt.date >= dt.date(2020, 3, 15)) & (x.OccurDate.dt.date <= dt.date(2022,6,30)),
        quadrant = lambda x: x.Neighborhood.apply(assign_quadrant))\
    .merge(hourly_weather.reset_index(drop=True),on=['OccurDate','OccurHour'],how='left')\
    .merge(daily_weather.reset_index(drop=True),on='OccurDate',how='left')\
    .assign(
        temp_bin = lambda x: x.temp_f.apply(temp_bin),
        rainy_day= lambda x: x.precipitation.apply(rainy_day),
        neigh_upper = lambda x: x.Neighborhood.str.upper()
    )\
    .merge(pdata_cleaned,left_on='neigh_upper',right_on='Neighborhood',how='left')\
    .drop(columns=['Neighborhood_y','neigh_upper'])\
    .assign(**{
    'Median household income ($)': lambda x: x['Median household income ($)'].astype(float)
})

pcrime_ml

In [ ]:
pcrime_ml_daily = pcrime_cleaned\
    .groupby(['OccurDate','Neighborhood'])\
    .agg(
        total_offenses=('OffenseCount','sum'),
        lat=('OpenDataLat','mean'),
        lon=('OpenDataLon','mean'))\
    .reset_index()\
    .assign(
        OccurDate = lambda x: pd.to_datetime(x.OccurDate),
        week=lambda x: x.OccurDate.dt.isocalendar().week,
        year=lambda x: x.OccurDate.dt.year,
        month=lambda x: x.OccurDate.dt.month,
        month_sin=lambda x: np.sin(2 * np.pi * x.month / 12),
        month_cos=lambda x: np.cos(2 * np.pi * x.month / 12),
        quarter=lambda x: x.OccurDate.dt.quarter,
        weekday=lambda x: x.OccurDate.dt.weekday,
        weekday_sin=lambda x: np.sin(2 * np.pi * x.weekday / 7),
        weekday_cos=lambda x: np.cos(2 * np.pi * x.weekday / 7),
        season_bin=lambda x: x['month'].apply(season_bin),
        weekday_bin=lambda x: x['weekday'].apply(weekday_bin),
        holiday=lambda x: x.OccurDate.isin(holidays),
        covid_time = lambda x: (x.OccurDate.dt.date >= dt.date(2020, 3, 15)) & (x.OccurDate.dt.date <= dt.date(2022,6,30)),
        quadrant = lambda x: x.Neighborhood.apply(assign_quadrant))\
    .merge(daily_weather.reset_index(drop=True),on='OccurDate',how='left')\
    .assign(
        temp_bin = lambda x: x.max_temp.apply(temp_bin),
        neigh_upper = lambda x: x.Neighborhood.str.upper()
    )\
    .merge(pdata_cleaned,left_on='neigh_upper',right_on='Neighborhood',how='left')\
    .drop(columns=['Neighborhood_y','neigh_upper'])\
    .assign(**{
    'Median household income ($)': lambda x: x['Median household income ($)'].astype(float)
})

pcrime_ml_daily

In [ ]:
daily_weather.head()

In [ ]:
pcrime_ml_daily_lag = pcrime_cleaned\
    .assign(OccurDate=lambda x: pd.to_datetime(x['OccurDate']).dt.date)\
    .groupby(['OccurDate','Neighborhood'])\
    .agg(
        total_offenses=('OffenseCount','sum'),
        lat=('OpenDataLat','mean'),
        lon=('OpenDataLon','mean'))\
    .reset_index()\
    .assign(
        OccurDate = lambda x: pd.to_datetime(x.OccurDate),
        week=lambda x: x.OccurDate.dt.isocalendar().week,
        year=lambda x: x.OccurDate.dt.year,
        month=lambda x: x.OccurDate.dt.month,
        month_sin=lambda x: np.sin(2 * np.pi * x.month / 12),
        month_cos=lambda x: np.cos(2 * np.pi * x.month / 12),
        quarter=lambda x: x.OccurDate.dt.quarter,
        weekday=lambda x: x.OccurDate.dt.weekday,
        weekday_sin=lambda x: np.sin(2 * np.pi * x.weekday / 7),
        weekday_cos=lambda x: np.cos(2 * np.pi * x.weekday / 7),
        season_bin=lambda x: x['month'].apply(season_bin),
        weekday_bin=lambda x: x['weekday'].apply(weekday_bin),
        holiday=lambda x: x.OccurDate.isin(holidays),
        covid_time = lambda x: (x.OccurDate.dt.date >= dt.date(2020, 3, 15)) & (x.OccurDate.dt.date <= dt.date(2022,6,30)),
        quadrant = lambda x: x.Neighborhood.apply(assign_quadrant))\
    .assign(
        lag_1 = lambda x: x.groupby('Neighborhood')['total_offenses'].shift(1),
        rolling_3 = lambda x: x.groupby('Neighborhood')['total_offenses'].shift(1).transform(lambda s: s.rolling(3).mean())
    )

pcrime_ml_daily_lag

In [ ]:
X = pcrime_ml_daily_lag.drop(columns=['total_offenses','OccurDate','month','weekday'],axis=1)
X = pd.get_dummies(X)
y = pcrime_ml_daily_lag['total_offenses']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train a Random Forest Regressor
xgb = XGBRegressor(
    max_depth=10,
    learning_rate=0.03,
    n_estimators=500,
    min_child_weight=25,
    subsample=.5,
    colsample_bytree=.6,
    random_state=42)
xgb.fit(X_train, y_train)

# Make predictions
y_pred = xgb.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test,y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f'RMSE: {rmse}')
print(f"R^2 Score: {r2}")

In [ ]:
X = pcrime_ml_daily.drop(columns=['total_offenses','OccurDate','month','weekday'],axis=1)
X = pd.get_dummies(X)
y = pcrime_ml_daily['total_offenses']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train a Random Forest Regressor
xgb = XGBRegressor(
    max_depth=10,
    learning_rate=0.03,
    n_estimators=500,
    min_child_weight=25,
    subsample=.5,
    colsample_bytree=.6,
    random_state=42)
xgb.fit(X_train, y_train)

# Make predictions
y_pred = xgb.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test,y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f'RMSE: {rmse}')
print(f"R^2 Score: {r2}")

In [ ]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score, root_mean_squared_error

In [ ]:
X = pcrime_ml.drop(columns=['total_offenses','OccurDate','OccurHour','month','weekday'],axis=1)
X = pd.get_dummies(X)
y = pcrime_ml['total_offenses']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train a Random Forest Regressor
xgb = XGBRegressor(
    max_depth=10,
    learning_rate=0.03,
    n_estimators=500,
    min_child_weight=25,
    subsample=.5,
    colsample_bytree=.6,
    random_state=42)
xgb.fit(X_train, y_train)

# Make predictions
y_pred = xgb.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test,y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f'RMSE: {rmse}')
print(f"R^2 Score: {r2}")

In [ ]:
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming 'xgb' is your trained XGBRegressor model and 'X' is your feature dataframe
feature_importances = pd.DataFrame({'Feature': X.columns, 'Importance': xgb.feature_importances_})
feature_importances = feature_importances.sort_values(by='Importance', ascending=False).head(20)

# Create a bar chart
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_importances)  # Top 15 features
plt.title('Top Feature Importances')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.show()

In [ ]:
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming 'xgb' is your trained XGBRegressor model and 'X' is your feature dataframe
feature_importances = pd.DataFrame({'Feature': X.columns, 'Importance': xgb.feature_importances_})
feature_importances = feature_importances.sort_values(by='Importance', ascending=False).head(20)

# Create a bar chart
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_importances)  # Top 15 features
plt.title('Top Feature Importances')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.show()

In [ ]:
lag_check = pcrime_ml_daily_lag[['OccurDate','Neighborhood','total_offenses','lag_1','rolling_3']].sort_values(['Neighborhood','OccurDate'])
lag_check

In [ ]:
dupes = pcrime_ml_daily_lag[pcrime_ml_daily_lag['Neighborhood'] == 'Brentwood-Darlington']
dupes = dupes[dupes['OccurDate'] == '2015-01-01']
print(dupes['OccurDate'].unique())